In [1]:
from pyspark import SparkContext as sc
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession, functions, types
from IPython.display import display

In [2]:
sc = sc(appName="attraction")
sqlContext = SQLContext(sc)
spark = SparkSession.builder.appName('attraction dataset combiner').getOrCreate()

In [3]:
det1 = 'outputs/attractions_details_batch1.json'
det2 = 'outputs/attractions_details_batch2.json'
rev1 = 'outputs/attractions_reviews_batch1.json'
rev2 = 'outputs/attractions_reviews_batch2.json'

In [4]:
det1_df = spark.read.json(det1)
det2_df = spark.read.json(det2)

In [5]:
det_joined = det1_df.union(det2_df)

if det1_df.count() + det2_df.count() == det_joined.count():
    print("combined successfully!")
else:
    print("error in dataframe union!")

combined successfully!


In [6]:
det_joined.write.parquet('outputs/attraction_details',mode='overwrite')

In [7]:
rev1_df = spark.read.json(rev1)
rev2_df = spark.read.json(rev2)

In [8]:
rev_joined = rev1_df.union(rev2_df)

if rev1_df.count() + rev2_df.count() == rev_joined.count():
    print("combined successfully!")
else:
    print("error in dataframe union!")

combined successfully!


In [9]:
rev_joined.write.parquet('outputs/attraction_reviews',mode='overwrite')

In [13]:
print(det_joined.count())
det_joined.show()

3637
+-------------+----------------+-------+--------------------+--------------------+------+----------------+------+
|attraction_id|            city|country|            location|                name| price|        province|rating|
+-------------+----------------+-------+--------------------+--------------------+------+----------------+------+
|            0|       vancouver| canada|{"lat":49.1978322...|vancouver_city_si...|  80.0|british_columbia|   4.5|
|            1|       vancouver| canada|{"lat":49.1978322...|vancouver_to_vict...| 210.0|british_columbia|   5.0|
|            2|        montreal| canada|{"lat":45.5001458...|quebec_city_and_m...| 115.0|          quebec|   4.5|
|            3|         toronto| canada|{"lat":43.6561507...|niagara_falls_day...| 169.0|         ontario|   5.0|
|            4|   niagara_falls| canada|{"lat":43.0857136...|best_of_niagara_f...| 158.0|         ontario|   5.0|
|            5|   niagara_falls| canada|{"lat":43.102436,...|niagara_falls_in_...|2

In [14]:
print(rev_joined.count())
rev_joined.show()

33925
+-------------+------+--------------------+------------------+--------------------+
|attraction_id|rating|              review|       review_date|                user|
+-------------+------+--------------------+------------------+--------------------+
|            0|   5.0|Another 'Dave' Gu...|    March 14, 2019|drew22perthaustralia|
|            0|   5.0|Fantastic way to ...|     March 1, 2019|              marc_h|
|            0|   5.0|This was a great ...| February 28, 2019|          maggiehand|
|            0|   5.0|All the main attr...| December 19, 2018|     catherine255066|
|            0|   5.0|Excellent Vancouv...| November 29, 2018|           gearjamkw|
|            0|   5.0|Highlight of our ...| November 12, 2018|              jkfran|
|            0|   5.0|It was worth taki...|  November 6, 2018|           cherylk16|
|            0|   5.0|Great Guide. Our ...|  November 2, 2018|             attydan|
|            0|   5.0|Better than Hop o...|  October 24, 2018|        